## Importing library

In [1]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy import signal
from enum import Enum
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import keras
import sys

DOMAINS = Enum('DOMAINS', 'Time_Nn Time_Ny Freq_Nn Freq_Ny Both_Nn Both_Ny', start=0)

SIGNALS = [
    "body_acc_x", "body_acc_y", "body_acc_z",
    "body_gyro_x", "body_gyro_y", "body_gyro_z",
    "total_acc_x", "total_acc_y", "total_acc_z"
]

ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

## Load data

In [2]:
from scipy.fftpack import fft

In [3]:
def _read_csv(subset, signal):
    if subset=='p':   return pd.read_csv(f'./Data_smartphone/{signal}.txt', delim_whitespace=True, header=None)
    elif subset=='a': return pd.read_csv(f'./Data_arduino/{signal}.txt', delim_whitespace=True, header=None)
    else:             return pd.read_csv(f'./Data_UCI/{signal}_{subset}.txt', delim_whitespace=True, header=None)

#------------------------------------------------------------------------------
def Load_data(subset):
    data = []
    for signal in SIGNALS:
        data.append( _read_csv(subset, signal).to_numpy())
    return data

def Load_EN_data(subset):
    data = []
    for signal in ['body_acc_en', 'body_gyro_en', 'total_acc_en']:
        data.append( _read_csv(subset, signal).to_numpy())
    return data

def Load_target(subset):
    return np.eye(6, dtype=int)[_read_csv(subset, 'y')[0]-1]

In [4]:
# UCI
raw_train, raw_test = Load_data('train'), Load_data('test')
raw_y_train, y_test = Load_target('train'), Load_target('test')

# Smartphone
raw_phone = Load_data('p')
raw_phone_en = Load_EN_data('p')
y_phone = Load_target('p')

# Arduino
raw_arduino = Load_data('a')
raw_arduino_en = Load_EN_data('p')
y_arduino = Load_target('a')

## Preprocess

In [5]:
expand = 'e' # n/e, n: 9 signals; e: 12 signals

shuffle_index = np.arange(raw_y_train.shape[0])
np.random.shuffle(shuffle_index)

In [6]:
def _normalization(data):
    normalized = np.empty(shape=data.shape)
    mean = np.mean(data, axis=1)
    std = np.std(data, axis=1)
    for row in range(data.shape[0]):
        if not std[row] == 0:
            for col in range(data.shape[1]):
                normalized[row][col] = (data[row][col] - mean[row]) / std[row]
    return normalized

def _FFT(data):
    return abs(fft(data))/128

def _shuffle(data, index):
    return data[index]

#------------------------------------------------------------------------------
def Expand(expand, data):
    new_data = []
    for i in range(9) : new_data.append(data[i])
    if expand=='e':
        for i in range(3): new_data.append(data[i])
    return new_data

def Normalize(data):
    new_data = []
    for i in range(len(data)):
        new_data.append( _normalization(data[i]) )
    return new_data

def Time_to_Freq(data):
    new_data = []
    for i in range(len(data)):
        new_data.append( _FFT(data[i]) )
    return new_data

def Reshape(data):
    data = np.transpose(data, (1,2,0))
    return data.reshape(data.shape[0], data.shape[1], data.shape[2], 1)

def Combine(time, freq):
    return np.transpose([time, freq], (2,3,1,0))

def Shuffle(index, data, channel, y):
    new_data = np.empty(shape=data.shape)
    for ch in range(channel):
        for i in range(len(index)):
            if y==True: new_data[i,:] = data[index[i],:]
            else: new_data[i,:,:,ch] = data[index[i],:,:,ch]
    return new_data

In [7]:
# UCI
time_Nn_train, time_Nn_test = Expand(expand, raw_train), Expand(expand, raw_test)
time_Ny_train, time_Ny_test = Normalize(time_Nn_train), Normalize(time_Nn_test)

# Smartphone
time_Nn_phone = Expand(expand, raw_phone)
time_Ny_phone = Normalize(time_Nn_phone)

# Arduino
time_Nn_arduino = Expand(expand, raw_arduino)
time_Ny_arduino = Normalize(time_Nn_arduino)

In [8]:
# UCI
freq_Nn_train, freq_Nn_test = Time_to_Freq(time_Nn_train), Time_to_Freq(time_Nn_test)
freq_Ny_train, freq_Ny_test = Time_to_Freq(time_Ny_train), Time_to_Freq(time_Ny_test)

# Smartphone
freq_Nn_phone = Time_to_Freq(time_Nn_phone)
freq_Ny_phone = Time_to_Freq(time_Ny_phone)

# Arduino
freq_Nn_arduino = Time_to_Freq(time_Nn_arduino)
freq_Ny_arduino = Time_to_Freq(time_Ny_arduino)

In [9]:
X_train = [Reshape(time_Nn_train),
           Reshape(time_Ny_train),
           Reshape(freq_Nn_train),
           Reshape(freq_Ny_train),
           Combine(time_Nn_train, freq_Nn_train),
           Combine(time_Ny_train, freq_Ny_train)]
X_test  = [Reshape(time_Nn_test),
           Reshape(time_Ny_test),
           Reshape(freq_Nn_test),
           Reshape(freq_Ny_test),
           Combine(time_Nn_test, freq_Nn_test),
           Combine(time_Ny_test, freq_Ny_test)]
X_phone = [Reshape(time_Nn_phone),
           Reshape(time_Ny_phone),
           Reshape(freq_Nn_phone),
           Reshape(freq_Ny_phone),
           Combine(time_Nn_phone, freq_Nn_phone),
           Combine(time_Ny_phone, freq_Ny_phone)]
X_arduino = [Reshape(time_Nn_arduino),
             Reshape(time_Ny_arduino),
             Reshape(freq_Nn_arduino),
             Reshape(freq_Ny_arduino),
             Combine(time_Nn_arduino, freq_Nn_arduino),
             Combine(time_Ny_arduino, freq_Ny_arduino)]


for index in range(len(X_train)):
    X_train[index] = Shuffle(shuffle_index, X_train[index], X_train[index].shape[3], False) 
y_train = Shuffle(shuffle_index, raw_y_train, 1, True)

In [10]:
print(X_train[0].shape)
print(y_train.shape)
print(X_test[0].shape)
print(y_test.shape)
print(X_phone[0].shape)
print(y_phone.shape)
print(X_arduino[0].shape)
print(y_arduino.shape)

(7352, 128, 12, 1)
(7352, 6)
(2947, 128, 12, 1)
(2947, 6)
(178, 128, 12, 1)
(178, 6)
(713, 128, 12, 1)
(713, 6)


## Function

In [11]:
def show_train_history(H):
    for length in range(len(H)):
        plt.plot(H[length].history['val_loss'])
    plt.title('Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
#     plt.legend(['BS:64', 'BS:64', 'BS:128', 'BS:128', 'BS:256', 'BS:256'], loc='upper right')
    plt.show()
    plt.clf()
    
    for length in range(len(H)):
        plt.plot(H[length].history['val_accuracy'])
    plt.title('Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
#     plt.legend(['BS:64', 'BS:64', 'BS:128', 'BS:128', 'BS:256', 'BS:256'], loc='upper right')
    plt.show()
    plt.clf()

## Neural network

In [12]:
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Flatten,LSTM
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import regularizers

In [13]:
def CNN_model(channel):
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(13,6), strides=(1,3),  input_shape=(128, 12, channel)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))

    model.add(Conv2D(filters=64, kernel_size=(13,3), strides=(1,1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))

    model.add(Conv2D(filters=128, kernel_size=(12,1), strides=(1,1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(2,1)))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(6, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
    model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

In [14]:
def training(X_train, y_train, validation_split, BS, Epochs):
    model = CNN_model(X_train.shape[3])
    history = model.fit(X_train, y_train, validation_split=validation_split, shuffle=True, batch_size=BS, epochs=Epochs)
    return model, history

In [15]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Time_Nn.value], y_train, 0.15, 300, 15 )

print("\nUCI data")
score = model.evaluate(X_test[DOMAINS.Time_Nn.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Time_Nn.value]))



Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 10s 2ms/step - loss: 1.1626 - accuracy: 0.6820 - val_loss: 5.3981 - val_accuracy: 0.5059
Epoch 2/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.4653 - accuracy: 0.8760 - val_loss: 0.3321 - val_accuracy: 0.9130
Epoch 3/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.2969 - accuracy: 0.9309 - val_loss: 0.3007 - val_accuracy: 0.9112
Epoch 4/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.2335 - accuracy: 0.9410 - val_loss: 0.2216 - val_accuracy: 0.9365
Epoch 5/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.1925 - accuracy: 0.9461 - val_loss: 0.2735 - val_accuracy: 0.9148
Epoch 6/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.1747 - accuracy: 0.9491 - val_loss: 0.1959 - val_accuracy: 0.9429
Epoch 7/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.1529 -

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,449,22,0,0,20
STANDING,0,158,369,1,1,3
WALKING,0,0,0,463,30,3
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,9,51,411


In [16]:
print("Smartphone data")
score = model.evaluate(X_phone[DOMAINS.Time_Nn.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Time_Nn.value]))

Smartphone data
178/178 [==============================] - 0s 538us/step
Phone accuracy:  0.0
Phone predict:


Pred,LAYING
True,
SITTING,139
WALKING,39


In [17]:
print("Arduino data")
score = model.evaluate(X_arduino[DOMAINS.Time_Nn.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Time_Nn.value]))

Arduino data
713/713 [==============================] - 0s 524us/step
Arduino accuracy:  0.0
Arduino predict:


Pred,WALKING_DOWNSTAIRS
True,
LAYING,126
SITTING,240
STANDING,148
WALKING,199


In [18]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Time_Ny.value], y_train, 0.15, 300, 15 )

print("\nUCI data:")
score = model.evaluate(X_test[DOMAINS.Time_Ny.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Time_Ny.value]))

Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 10s 2ms/step - loss: 1.3002 - accuracy: 0.6335 - val_loss: 0.7166 - val_accuracy: 0.7851
Epoch 2/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.4840 - accuracy: 0.8491 - val_loss: 0.3976 - val_accuracy: 0.8676
Epoch 3/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.3413 - accuracy: 0.8937 - val_loss: 0.3127 - val_accuracy: 0.8957
Epoch 4/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.2391 - accuracy: 0.9347 - val_loss: 0.2645 - val_accuracy: 0.9193
Epoch 5/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.1975 - accuracy: 0.9448 - val_loss: 0.2733 - val_accuracy: 0.9102
Epoch 6/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.1663 - accuracy: 0.9554 - val_loss: 0.2659 - val_accuracy: 0.9121
Epoch 7/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.1318 - 

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,422,90,24,1,0,0
SITTING,19,456,15,0,1,0
STANDING,14,84,432,1,1,0
WALKING,7,0,2,475,11,1
WALKING_DOWNSTAIRS,0,0,0,0,415,5
WALKING_UPSTAIRS,0,36,0,0,9,426


In [19]:
print("Smartphone data:")
score = model.evaluate(X_phone[DOMAINS.Time_Ny.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Time_Ny.value]))

Smartphone data:
178/178 [==============================] - 0s 465us/step
Phone accuracy:  0.33707866072654724
Phone predict:


Pred,LAYING,SITTING,STANDING,WALKING_DOWNSTAIRS
True,,,,
SITTING,64,60,14,1
WALKING,3,28,7,1


In [20]:
print("Arduino data:")
score = model.evaluate(X_arduino[DOMAINS.Time_Ny.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Time_Ny.value]))

Arduino data:
713/713 [==============================] - 0s 512us/step
Arduino accuracy:  0.29593268036842346
Arduino predict:


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,27,63,33,0,3,0
SITTING,91,123,26,0,0,0
STANDING,30,64,50,2,2,0
WALKING,24,104,42,11,16,2


In [21]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Freq_Nn.value], y_train, 0.15, 300, 15 )

print("\nUCI data:")
score = model.evaluate(X_test[DOMAINS.Freq_Nn.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Freq_Nn.value]))

Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 10s 2ms/step - loss: 2.1786 - accuracy: 0.3732 - val_loss: 8.9110 - val_accuracy: 0.1659
Epoch 2/15
6249/6249 [==============================] - 9s 1ms/step - loss: 1.2496 - accuracy: 0.5183 - val_loss: 5.2684 - val_accuracy: 0.2022
Epoch 3/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.7906 - accuracy: 0.6623 - val_loss: 4.5488 - val_accuracy: 0.3264
Epoch 4/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.6276 - accuracy: 0.7225 - val_loss: 4.8786 - val_accuracy: 0.4134
Epoch 5/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.5090 - accuracy: 0.7638 - val_loss: 0.5055 - val_accuracy: 0.7425
Epoch 6/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.4284 - accuracy: 0.7996 - val_loss: 0.4487 - val_accuracy: 0.7960
Epoch 7/15
6249/6249 [==============================] - 9s 1ms/step - loss: 0.3819 -

In [22]:
print("Smartphone data:")
score = model.evaluate(X_phone[DOMAINS.Freq_Nn.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Freq_Nn.value]))

Smartphone data:
178/178 [==============================] - 0s 577us/step
Phone accuracy:  0.1910112351179123
Phone predict:


Pred,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,
SITTING,2,135,2,0,0
WALKING,0,0,32,6,1


In [23]:
print("Arduino data:")
score = model.evaluate(X_arduino[DOMAINS.Freq_Nn.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Freq_Nn.value]))

Arduino data:
713/713 [==============================] - 0s 492us/step
Arduino accuracy:  0.38849928975105286
Arduino predict:


Pred,LAYING,SITTING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,
LAYING,126,0,0,0,0
SITTING,234,0,6,0,0
STANDING,136,2,4,0,6
WALKING,0,0,151,48,0


In [24]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Freq_Ny.value], y_train, 0.15, 300, 15 )

print("\nUCI data:")
score = model.evaluate(X_test[DOMAINS.Freq_Ny.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Freq_Ny.value]))

Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 11s 2ms/step - loss: 2.1390 - accuracy: 0.4458 - val_loss: 9.0818 - val_accuracy: 0.2611
Epoch 2/15
6249/6249 [==============================] - 10s 2ms/step - loss: 1.1064 - accuracy: 0.6230 - val_loss: 3.0972 - val_accuracy: 0.2892
Epoch 3/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.8427 - accuracy: 0.7118 - val_loss: 0.9092 - val_accuracy: 0.6745
Epoch 4/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.7287 - accuracy: 0.7425 - val_loss: 0.6283 - val_accuracy: 0.7652
Epoch 5/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.6113 - accuracy: 0.7736 - val_loss: 0.5645 - val_accuracy: 0.7833
Epoch 6/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.5581 - accuracy: 0.7953 - val_loss: 0.7373 - val_accuracy: 0.7271
Epoch 7/15
6249/6249 [==============================] - 11s 2ms/step - loss: 0.522

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,502,21,13,0,1,0
SITTING,60,298,132,0,0,1
STANDING,36,39,457,0,0,0
WALKING,4,1,2,428,36,25
WALKING_DOWNSTAIRS,5,0,0,1,406,8
WALKING_UPSTAIRS,6,7,7,1,22,428


In [25]:
print("Smartphone data:")
score = model.evaluate(X_phone[DOMAINS.Freq_Ny.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Freq_Ny.value]))

Smartphone data:
178/178 [==============================] - 0s 504us/step
Phone accuracy:  0.11797752976417542
Phone predict:


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_UPSTAIRS
True,,,,,
SITTING,69,14,56,0,0
WALKING,0,28,1,7,3


In [26]:
print("Arduino data:")
score = model.evaluate(X_arduino[DOMAINS.Freq_Ny.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Freq_Ny.value]))

Arduino data:
713/713 [==============================] - 0s 525us/step
Arduino accuracy:  0.3814866840839386
Arduino predict:


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,76,3,47,0,0,0
SITTING,86,153,1,0,0,0
STANDING,38,72,38,0,0,0
WALKING,36,143,0,5,4,11


In [27]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Both_Nn.value], y_train, 0.15, 300, 15 )

print("\nUCI data:")
score = model.evaluate(X_test[DOMAINS.Both_Nn.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Both_Nn.value]))

Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.9842 - accuracy: 0.7315 - val_loss: 7.7549 - val_accuracy: 0.4841
Epoch 2/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.4436 - accuracy: 0.8864 - val_loss: 0.4906 - val_accuracy: 0.8713
Epoch 3/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.2672 - accuracy: 0.9342 - val_loss: 0.3247 - val_accuracy: 0.8776
Epoch 4/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.2044 - accuracy: 0.9450 - val_loss: 0.2204 - val_accuracy: 0.9338
Epoch 5/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.1753 - accuracy: 0.9482 - val_loss: 0.1918 - val_accuracy: 0.9365
Epoch 6/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.1582 - accuracy: 0.9490 - val_loss: 0.1986 - val_accuracy: 0.9438
Epoch 7/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.1466 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,509,1,0,0,0,27
SITTING,0,376,92,0,0,23
STANDING,0,87,409,1,0,35
WALKING,0,0,0,463,29,4
WALKING_DOWNSTAIRS,0,0,0,0,417,3
WALKING_UPSTAIRS,0,0,0,1,29,441


In [28]:
print("Smartphone data:")
score = model.evaluate(X_phone[DOMAINS.Both_Nn.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Both_Nn.value]))

Smartphone data:
178/178 [==============================] - 0s 566us/step
Phone accuracy:  0.0
Phone predict:


Pred,LAYING,WALKING_DOWNSTAIRS
True,,
SITTING,139,0
WALKING,9,30


In [29]:
print("Arduino data:")
score = model.evaluate(X_arduino[DOMAINS.Both_Nn.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Both_Nn.value]))

Arduino data:
713/713 [==============================] - 0s 522us/step
Arduino accuracy:  0.0
Arduino predict:


Pred,WALKING_DOWNSTAIRS
True,
LAYING,126
SITTING,240
STANDING,148
WALKING,199


In [30]:
# for domain in DOMAINS:
model, history = training( X_train[DOMAINS.Both_Ny.value], y_train, 0.15, 300, 15 )

print("\nUCI data:")
score = model.evaluate(X_test[DOMAINS.Both_Ny.value], y_test)
print("Test accuracy: ", score[1])
print('Test predict:')
confusion_matrix(y_test, model.predict(X_test[DOMAINS.Both_Ny.value]))

Train on 6249 samples, validate on 1103 samples
Epoch 1/15
6249/6249 [==============================] - 11s 2ms/step - loss: 1.1985 - accuracy: 0.6675 - val_loss: 0.7881 - val_accuracy: 0.7851
Epoch 2/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.4489 - accuracy: 0.8691 - val_loss: 0.3924 - val_accuracy: 0.8758
Epoch 3/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.3001 - accuracy: 0.9160 - val_loss: 0.2894 - val_accuracy: 0.9121
Epoch 4/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.2150 - accuracy: 0.9435 - val_loss: 0.2518 - val_accuracy: 0.9202
Epoch 5/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.1710 - accuracy: 0.9565 - val_loss: 0.2329 - val_accuracy: 0.9275
Epoch 6/15
6249/6249 [==============================] - 9s 2ms/step - loss: 0.1246 - accuracy: 0.9720 - val_loss: 0.2237 - val_accuracy: 0.9257
Epoch 7/15
6249/6249 [==============================] - 10s 2ms/step - loss: 0.1015 -

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,433,69,35,0,0,0
SITTING,20,447,24,0,0,0
STANDING,7,70,455,0,0,0
WALKING,0,1,16,469,9,1
WALKING_DOWNSTAIRS,6,0,0,3,408,3
WALKING_UPSTAIRS,0,47,1,0,2,421


In [31]:
print("Smartphone data:")
score = model.evaluate(X_phone[DOMAINS.Both_Ny.value], y_phone)
print("Phone accuracy: ", score[1])
print('Phone predict:')
confusion_matrix(y_phone, model.predict(X_phone[DOMAINS.Both_Ny.value]))

Smartphone data:
178/178 [==============================] - 0s 560us/step
Phone accuracy:  0.17977528274059296
Phone predict:


Pred,LAYING,SITTING,STANDING
True,,,
SITTING,47,32,60
WALKING,4,30,5


In [32]:
print("Arduino data:")
score = model.evaluate(X_arduino[DOMAINS.Both_Ny.value], y_arduino)
print("Arduino accuracy: ", score[1])
print('Arduino predict:')
confusion_matrix(y_arduino, model.predict(X_arduino[DOMAINS.Both_Ny.value]))

Arduino data:
713/713 [==============================] - 0s 534us/step
Arduino accuracy:  0.3716689944267273
Arduino predict:


Pred,LAYING,SITTING,STANDING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,
LAYING,8,28,88,2,0
SITTING,26,161,53,0,0
STANDING,2,50,96,0,0
WALKING,98,33,19,47,2
